In [24]:
#NEEDED LIBRARIES
from os import path
from requests import get,session
from bs4 import BeautifulSoup as bs 
from selenium import webdriver
import json 
import re
from time import sleep

In [25]:
#JUST WEBSITE NAMES
twitter_url = "https://twitter.com"
insta_url = "https://www.instagram.com"
fb_url = "https://m.facebook.com/public"

In [28]:
#INSTA INFO
try:
    insta_username = input("Instagram Username:")
    insta_final = f"{insta_url}/{insta_username}/"
    user_page = get(insta_final)
    insta_soup = bs(user_page.text,'lxml')
    scripts = insta_soup.find('script', type="text/javascript", text=re.compile('window._sharedData'))
    stringified_json = scripts.text.replace('window._sharedData = ', '')[:-1]
    final_data = json.loads(stringified_json)['entry_data']['ProfilePage'][0]
    insta_username = final_data['graphql']['user']['username']
    data_file = f"{insta_username}'s_insta_data.json" if not path.isfile(f"{insta_username}'s_insta_data.json") else f"{insta_username}'s_insta_data_1.json"
    json.dump(final_data, open(data_file,"w+"), indent=4)
    print("\nData dumped!")
except:
    print(f"Sorry {insta_username} is not a valid instagram username")

Instagram Username:_cosec_

Data dumped!


In [29]:
def download(username,site_name,dp_url):
    count=0
    filename = f"{username}'s_{site_name}_dp.jpg"
    while True:
        if not path.isfile(filename):
            with open(filename, 'wb+') as handle:
                response = get(dp_url, stream=True)
                if not response.ok:
                    print(response)
                for block in response.iter_content(1024):
                    if not block:
                        break
                    handle.write(block)
        else:
            count+=1
            filename = f"{username}'s_{site_name}_dp_{count}.jpg"
            continue
        break

In [30]:
#DOWNLOAD INSTA DP
dp_url = final_data['graphql']['user']['profile_pic_url_hd']
filename = f"{insta_username}'s_insta_dp.jpg"
count=0
download(insta_username,"insta",dp_url)
print("DP downloaded!")

DP downloaded!


In [10]:
#FACEBOOK DP
full_name = final_data['graphql']['user']['full_name']
fb_search_url = f"{fb_url}/{full_name}"
fb_search_results = get(fb_search_url)
fb_soup = bs(fb_search_results.text,'lxml')
fb_dp = fb_soup.find('img',{"alt":f"{full_name}"})
print(list(i["src"]for i in fb_dp))

[]


In [22]:
#TWITTER INFO
username=input("Please enter a twitter username: ")
page = get(f"https://twitter.com/{username}")
try:
    html = bs(page.text,'html.parser')
except KeyError:
    raise ValueError(
        f'Oops! Either "{username}" does not exist or is private.')
except ParserError:
    pass
# TODO: Check what kind of exception raising if no location
location = html.find("span",{"class":"ProfileHeaderCard-locationText u-dir"}).text
birthday = html.find("span",{"class":"ProfileHeaderCard-birthdateText u-dir"}).text.strip()
if birthday:
    birthday = birthday.replace('Born ', '')
else:
    birthday = None
profile_photo = html.find("img",{"class":"ProfileAvatar-image"}).attrs['src']
page_title = html.find('title').text
name = page_title[:page_title.find('(')].strip()
biography = html.find("p",{"class":"ProfileHeaderCard-bio u-dir"}).text
website = html.find("a",{"class":"u-textUserColor","rel":"me nofollow noopener"}).text.strip()
print(dict(
    name = name,
    username = username,
    birthday = birthday,
    biography = biography,
    website = website,
    profile_photo = profile_photo,
))

Please enter a twitter username: realDonaldTrump
{'name': 'Donald J. Trump', 'username': 'realDonaldTrump', 'birthday': None, 'biography': '45th President of the United States of America', 'website': 'Instagram.com/realDonaldTrump', 'profile_photo': 'https://pbs.twimg.com/profile_images/874276197357596672/kUuht00m_400x400.jpg'}
